## Оптимизация выполнения кода, векторизация, Numba

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [ ]:
import numpy as np
import numba
A = np.random.randint (0,1000,size=(1_000_000))

@numba.njit
def f1(A):
  acc,cnt = 0,0
  for ai in A:
    bi=ai+100
    acc+=bi
    cnt+=1
  return acc/cnt


%timeit f1(A)

%lprun -f f1 f1(A)

The slowest run took 264.15 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 438 µs per loop


UsageError: Line magic function `%lprun` not found.


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [ ]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)),
                  columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)
#lettersS= ['a', 'b', 'c', 'd', 'e']
letters=letters[0:5]
print (letters)

#@numba.njit
def g(df, letters):
    
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

%timeit g(df,letters ) #
%lprun -f g  g(df, letters)

['a', 'b', 'c', 'd', 'e']
1 loop, best of 5: 667 ms per loop


## Лабораторная работа 3

In [ ]:
!pip install line_profiler

     |████████████████████████████████| 63 kB 2.4 MB/s 


In [ ]:
%load_ext line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [3]:
#С использованием метода DataFrame.iterrows исходной таблицы;

import numpy as np
import pandas as pd




recipes  = pd.read_csv('/content/recipes_sample.csv', sep =',', names=["name","id","minutes","contributor_id","submitted","n_steps","description","n_ingredients"])
reviews = pd.read_csv('/content/reviews_sample.csv', sep =',', names=["ID","user_id","recipe_id","date","rating","review"], skiprows=1)
reviews['rating']=pd.to_numeric(reviews['rating'])

#reviews['date'] = pd.to_datetime(reviews['date'])
#reviews["date">"2009-12-30" and "date"<"2011-01-01" ]


def task_A(reviews,arr):
  #reviews=reviews.loc[(reviews['date'] >= '2009-31-12') & (reviews['date'] < '2011-01-01')]
  for i , rows in reviews.iterrows():
    arr=np.append(arr,int(rows[4]))
  return (np.mean(arr))

arr=np.array([])

%timeit print(task_A(reviews,arr))

%lprun -f  task_A  task_A(reviews, arr)

4.410802235271832
4.410802235271832
4.410802235271832
4.410802235271832
4.410802235271832
4.410802235271832
1 loop, best of 5: 24.9 s per loop


UsageError: Line magic function `%lprun` not found.


In [ ]:
#С использованием метода DataFrame.iterrows таблицы, в которой сохранены только отзывы за 2010 год; 

import numpy as np
import pandas as pd

recipes  = pd.read_csv('/content/recipes_sample.csv', sep =',', names=["name","id","minutes","contributor_id","submitted","n_steps","description","n_ingredients"])
reviews = pd.read_csv('/content/reviews_sample.csv', sep =',', names=["ID","user_id","recipe_id","date","rating","review"], skiprows=1)
reviews['rating']=pd.to_numeric(reviews['rating'])
reviews=reviews.loc[(reviews['date'] >= '2009-31-12') & (reviews['date'] < '2011-01-01')]

def task_B(reviews,arr):
  #reviews=reviews.loc[(reviews['date'] >= '2009-31-12') & (reviews['date'] < '2011-01-01')]
  for i , rows in reviews.iterrows():
    arr=np.append(arr,int(rows[4]))
  return (np.mean(arr))

arr=np.array([])

%timeit print(task_B(reviews,arr))

%lprun -f  task_B  task_B(reviews,arr)



4.4544402182900615
4.4544402182900615
4.4544402182900615
4.4544402182900615
4.4544402182900615
4.4544402182900615
1 loop, best of 5: 1.29 s per loop


In [ ]:
#С использованием метода Series.mean.

import numpy as np
import pandas as pd

recipes  = pd.read_csv('/content/recipes_sample.csv', sep =',', names=["name","id","minutes","contributor_id","submitted","n_steps","description","n_ingredients"])
reviews = pd.read_csv('/content/reviews_sample.csv', sep =',', names=["ID","user_id","recipe_id","date","rating","review"])


def Task_C(reviews):
  return (reviews.mean())

%timeit print(Task_C(reviews))

%lprun -f  Task_C  Task_C(reviews)

ParserError: ignored

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?



3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [ ]:
# def get_word_reviews_count(df):
#     word_reviews = {}
#     for _, row in df.dropna(subset=['review']).iterrows():
#         recipe_id, review = row['recipe_id'], row['review']
#         words = review.split(' ')
#         for word in words:
#             if word not in word_reviews:
#                 word_reviews[word] = []
#             word_reviews[word].append(recipe_id)
    
#     word_reviews_count = {}
#     for _, row in df.dropna(subset=['review']).iterrows():
#         review = row['review']
#         words = review.split(' ')
#         for word in words:
#             word_reviews_count[word] = len(word_reviews[word])
#     return word_reviews_count


import numba
import pandas as pd


def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

reviews = pd.read_csv('/content/reviews_sample.csv', sep =',', names=["ID","user_id","recipe_id","date","rating","review"])
#get_word_reviews_count(reviews)

%lprun -f  get_word_reviews_count  get_word_reviews_count(reviews)

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [ ]:
#Без использования векторизованных операций и методов массивов numpy и без использования numba
import numba
import pandas as pd

reviews = pd.read_csv('/content/reviews_sample.csv', sep =',', names=["ID","user_id","recipe_id","date","rating","review"])
reviews = pd.read_csv('/content/reviews_sample.csv', sep =',', names=["ID","user_id","recipe_id","date","rating","review"], skiprows=1)
rating = reviews[['rating']].values.tolist()
print (len(rating))
mes=reviews.groupby('ID')['rating'].agg(['mean'])
mes=mes.values.tolist()

#@numba.njit
def task_1D(mes,rating):
  return (mean_absolute_error(rating, mes)*100)

task_1D(mes,rating)


In [ ]:
#Без использования векторизованных операций и методов массивов numpy, но с использованием numba
import numba
import pandas as pd
from sklearn.metrics import mean_absolute_error


reviews = pd.read_csv('/content/reviews_sample.csv', sep =',', names=["ID","user_id","recipe_id","date","rating","review"])

reviews = pd.read_csv('/content/reviews_sample.csv', sep =',', names=["ID","user_id","recipe_id","date","rating","review"], skiprows=1)
rating = reviews[['rating']].values.tolist()
print (len(rating))
mes=reviews.groupby('ID')['rating'].agg(['mean'])
mes=mes.values.tolist()

#@numba.njit
def task_2D(mes,rating):
  return (mean_absolute_error(rating, mes)*100)

task_2D(mes,rating)

126696


96.54132727157922

In [ ]:
#С использованием векторизованных операций и методов массивов numpy, но без использования numba
import numba
import pandas as pd

reviews = pd.read_csv('/content/reviews_sample.csv', sep =',', names=["ID","user_id","recipe_id","date","rating","review"], skiprows=1)
rating = reviews[['rating']].to_numpy()
print (len(rating))
mes=reviews.groupby('ID')['rating'].agg(['mean'])
mes=mes.to_numpy()

def task_3D(mes,rating):
  np.mean(np.abs((rating - mes) / rating)) * 100

task_3D(mes,rating)

126696


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [ ]:
#C использованием векторизованных операций и методов массивов numpy и numba
import numba
import pandas as pd
#import numpy as np

reviews = pd.read_csv('/content/reviews_sample.csv', sep =',', names=["ID","user_id","recipe_id","date","rating","review"], skiprows=1)
#reviews['rating']=pd.to_numeric(reviews['rating'])
#means=reviews.groupby('ID').mean()
#print (means)
#print (reviews)

#res=reviews.groupby(['recipe_id'])['rating'].describe()[[ 'mean']]
#means=res[['mean']]

rating = reviews[['rating']].to_numpy()
print (len(rating))
mes=reviews.groupby('ID')['rating'].agg(['mean'])
mes=mes.to_numpy()

@numba.njit
def task_4D(mes,rating):
  np.mean(np.abs((rating - mes) / rating)) * 100

task_4D(mes,rating)
  #np.mean(np.abs((actual - pred) / actual)) * 100

126696
